In [8]:
import os
import pandas as pd
import regex as re
ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))

In [9]:
sources = ["AP", "Fox", "CNN", "ABC", "CBS", "NYT", "Mirror", "Reuters", "Express", "Guardian", "DailyMail"]
for source in sources:
    data = pd.read_csv(os.path.join(ROOT_DIR,"data",f"{source}.csv"),parse_dates=["Date"],index_col="Date")
    data = data.sort_index().loc["2022-01-01":"2022-12-31"]
    data.to_csv(os.path.join(ROOT_DIR,"data",f"{source}.csv"))

In [10]:
def query_string_gen(file):
    l=[]

    with open(os.path.join(ROOT_DIR,"results_0",f"{file}_TopicsWords.txt"), 'r') as file:
        lines = file.readlines()
    for line in lines:
        #print(line[0:1])
        if line[0:1] != "#":
            l.append(re.search(r'\d+', line).group())
    l = list(dict.fromkeys(l))

    string = ""
    for i,j in enumerate(l):
        if i==len(l)-1:
            string += f"Topic == {j}"
        else:
            string += f"Topic == {j} | "
    return string

def filter_corpus(source):
    data = pd.read_csv(os.path.join(ROOT_DIR,"results_0",f"{source}_Results.csv"))
    data_keep = data.query(query_string_gen(source))
    data_removed = pd.concat([data, data_keep])
    data_removed = data_removed.drop_duplicates(keep=False)
    data_keep = data_keep.drop(columns=["Topic"])
    data_keep.to_csv(os.path.join(ROOT_DIR,"data_filtered",f"{source}.csv"),index=False)
    data_removed.to_csv(os.path.join(ROOT_DIR,"data_filtered","data_removed",f"{source}.csv"),index=False)


In [11]:
corpora = ["ABC","AP","CBS","CNN","DailyMail","Express","Fox","Guardian", "Mirror", "NYT", "Reuters"] #Mirror, NYT, Reuters
for corpus in corpora:
    filter_corpus(corpus)

In [16]:
def unite_sources():
    total = 0
    sources = ["ABC","AP","CBS","CNN","DailyMail","Express","Fox","Guardian", "Mirror", "NYT", "Reuters"]
    data = [pd.read_csv(os.path.join(ROOT_DIR,"results_0",f"{i}_Results.csv")) for i in sources]
    result = pd.DataFrame(columns=["Source", "Date", "URL", "Title", "Text"])
    print("-> Current Dataset:")
    for i, j in zip(data, sources):
        if j == "All":
            continue
        i["Source"] = j
        print(f"{j}: {len(i)} Articles")
        total += len(i)
        result = pd.concat([result, i])
    result = result.drop_duplicates(subset=["Text"])
    result = result.set_index("Date")
    result = result.sort_index(ascending=False)
    result = result.drop(columns=["Topic"])
    result.to_csv(os.path.join(ROOT_DIR, "data_filtered", "All.csv"), index=True)
    print(f"-> Saved CSV with {total} articles.\n")

In [17]:
unite_sources()

-> Current Dataset:
ABC: 885 Articles
AP: 5608 Articles
CBS: 3306 Articles
CNN: 2367 Articles
DailyMail: 7575 Articles
Express: 9607 Articles
Fox: 4300 Articles
Guardian: 4672 Articles
Mirror: 2136 Articles
NYT: 2387 Articles
Reuters: 14825 Articles
-> Saved CSV with 57668 articles.

